In [1]:
!pip install metaflow==2.13.9 pandas==2.2.3 numpy==1.26.4 scikit-learn==1.6.1 optuna==4.2.0

In [2]:
# Set username for workflows
import os
os.environ["USERNAME"] = "nacho"

In [3]:
%%writefile metaflow_trainingflow.py
from metaflow import FlowSpec, Parameter, step

class TrainingFlow(FlowSpec):
    # Define the parameters for the flow
    max_depth = Parameter('max_depth', default=2, help='Max depth of the random forest classifier')
    n_estimators = Parameter('n_estimators', default=100, help='Number of estimators for the random forest classifier')
    random_state = Parameter('random_state', default=0, help='Random state for the random forest classifier')

    @step
    def start(self):
        # Start the flow
        self.next(self.ingest_data)

    @step
    def ingest_data(self):
        from sklearn.datasets import load_iris

        # Load the iris dataset
        iris = load_iris()

        #pylint: disable=no-member
        self.X = iris.data
        self.y = iris.target
        #pylint: enable=no-member

        self.next(self.split_data)

    @step
    def split_data(self):
        # Split the data into train and test
        from sklearn.model_selection import train_test_split
        X_train, X_test, y_train, y_test = train_test_split(self.X, self.y, test_size=0.2, random_state=self.random_state)

        self.X_train = X_train
        self.X_test = X_test
        self.y_train = y_train
        self.y_test = y_test

        self.next(self.train)

    @step
    def train(self):
        # Train the model
        from sklearn.ensemble import RandomForestClassifier

        # Create the random forest classifier
        self.rf = RandomForestClassifier(max_depth=self.max_depth, n_estimators=self.n_estimators, random_state=self.random_state)
        # Train the model
        self.rf.fit(self.X_train, self.y_train)

        self.next(self.show_metrics)

    @step
    def show_metrics(self):
        # Print some metrics
        from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

        y_pred = self.rf.predict(self.X_test)
        accuracy = accuracy_score(self.y_test, y_pred)
        report = classification_report(self.y_test, y_pred)
        confusion = confusion_matrix(self.y_test, y_pred)

        print(f'Accuracy: {accuracy}')
        print(f'Report:\n{report}')
        print(f'Confusion matrix:\n{confusion}')

        self.next(self.register_model)

    @step
    def register_model(self):
        # Save the model in a pickle file in local storage
        import pickle
        with open('model.pkl', 'wb') as f:
            pickle.dump(self.rf, f)

        self.next(self.end)

    @step
    def end(self):
        pass

if __name__ == '__main__':
    TrainingFlow()

Writing metaflow_trainingflow.py


In [4]:
!python metaflow_trainingflow.py run --max_depth 2 --n_estimators 100 --random_state 0

Metaflow 2.13.9 executing TrainingFlow for user:nacho
Creating local datastore in current directory (/content/.metaflow)
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint not found, so extra checks are disabled.
2025-04-01 14:08:42.895 Workflow starting (run-id 1743516522893715):
2025-04-01 14:08:42.913 [1743516522893715/start/1 (pid 3207)] Task is starting.
2025-04-01 14:08:43.932 [1743516522893715/start/1 (pid 3207)] Task finished successfully.
2025-04-01 14:08:43.941 [1743516522893715/ingest_data/2 (pid 3213)] Task is starting.
2025-04-01 14:08:46.983 [1743516522893715/ingest_data/2 (pid 3213)] Task finished successfully.
2025-04-01 14:08:46.987 [1743516522893715/split_data/3 (pid 3233)] Task is starting.
2025-04-01 14:08:48.702 [1743516522893715/split_data/3 (pid 3233)] Task finished successfully.
2025-04-01 14:08:48.705 [1743516522893715/train/4 (pid 3248)] Task is starting.
2025-04-01 14:08:51.523 [1743516522893715/train/4 (pid 3248)] Task finished su